**본 프로젝트의 배경은 다음과 같다.**

올해 1월부터 7월까지 스미싱 범죄 건수는 17만6220건으로 지난해 같은 기간(14만5093건)에 비해 21.5% 증가했습니다.

특히 최근 교묘하고 지능적인 스미싱 문자 패턴으로 인해 고객들의 피해가 증가하고 있습니다. 이를 방지하기 위해 kb 금융그룹과 KISA는 데이코너들에게 도움을 요청합니다.

[14회 금융문자 분석 경진대회](https://newfront.dacon.io/competitions/official/235401/overview/description/)

주최 : KB금융지주, DACON, KISA(한국인터넷진흥원)

주관 : DACON

# content
1. library & data
2. 데이터 전처리
3. 클래스 불균형 맞추기
4. Tokenizing
5. 검증용 데이터 나누기
6. 다단어 표현 추출
7. word2vec
8. 정수 인코딩, 패딩
9. 모델

# library & data

### library

In [0]:
import numpy as np                                      # 행렬 계산
import pandas as pd                                     # 데이터 프레임

from sklearn.model_selection import train_test_split    # validation dataset
from keras_preprocessing.text import Tokenizer          # 전처리
from keras_preprocessing.sequence import pad_sequences  # 전처리

from collections import Counter                         # 클래스 별 개수 구하기

from imblearn.under_sampling import RandomUnderSampler  # 샘플링
from imblearn.over_sampling import RandomOverSampler, SVMSMOTE    # 샘플링

from gensim.models import Word2Vec
from gensim.models.phrases import Phraser
from gensim.models import Phrases

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### 파일

코랩에서 드라이브 마운트 기능을 제공한다.

In [0]:
%cd /content/drive/My Drive/smishing/

/content/drive/My Drive/smishing


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('public_test.csv')
submission = pd.read_csv('submission2.csv', index_col='id')

# 데이터 전처리

In [0]:
# 정규표현식으로 필요없는 문자 삭제
train['text'] = train['text'].str.replace('[\\WX]', ' ')
test['text'] = test['text'].str.replace('[\\WX]', ' ')

In [0]:
# 전처리 확인
train.head(3)

,id,year_month,text,smishing
0,0,2017-01,은행성산 팀장입니다 행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요 은행 진월동VIP라운지 올림,0
2,2,2017-01,안녕하십니까 고객님 은행입니다 금일 납부하셔야 할 금액은 153600원 입니...,0


# 클래스 불균형 맞추기

클래스 불균형을 해소하기 위한 방법은 크게 2가지로 나누어진다.


**1.   Undersampling**

다수 클래스의 데이터 개수를 소수 클래스 데이터 개수에 맞추기 위해 무작위로 다수 클래스 데이터를 추출하는 것을 말한다.

**2.   Oversampling**

소수 클래스 데이터의 개수를 다수 클래스 데이터 개수에 맞추기 위해 소수 클래스 데이터를 변형하여 추가하는 등의 과정을 통해 소수 클래스 데이터 개수를 늘리는 것을 말한다.<br><br>
oversampling은 데이터 양을 늘릴 순 있지만 예측에 방해가 되는 데이터를 만들 수 있기 때문에 undersampling을 사용했다.

![대체 텍스트](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FcvKJgV%2FbtqCoQ17v69%2FXl6Z7Nd1LieWsrKw1cfqL1%2Fimg.png)<br><br>
스미싱 문자는 전체의 6% 정도밖에 되지 않는다. 그래서 스미싱 문자는 모두 사용하고 일반 문자는 스미싱 문자가 전체의 20%가 될 수 있도록 랜덤하게 선택한다.

### Undersampling

In [0]:
# 변수 설명
# random_state : random state
# minor : 소수 클래스 비율
# major : 다수 클래스 비율
# X : 'smishing'열을 제외한 나머지 데이터
# y : 'smishing' 열

def undersampling(random_state, minor, major):
  X = train.drop('smishing', axis=1)
  y = train['smishing']
  rus = RandomUnderSampler(random_state=random_state, sampling_strategy=minor/major)
  x_sampled, y_sampled = rus.fit_resample(X, y)
  col = train.columns
  train_sample = pd.DataFrame(np.hstack((x_sampled, y_sampled.reshape(-1, 1))), columns=col)
  return train_sample

train_sample = undersampling(2020, 2, 8)
train_sample.head()

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


,id,year_month,text,smishing
0,191613,2017-10,봉덕동 계장입니다한주의중반수요일따뜻한햇살과함께즐거운하루보내세요,0
1,323818,2018-11,광고 개인형IRP는 선택이 아닌 필수 고객님 연말정산 세액공제로 13월의 ...,0
2,224978,2018-02,고객님활기 넘치는 무술년 한해 건강하게 보내시고 따뜻한 고향의 정을 가족과 ...,0
3,73014,2017-04,웃는얼굴은 하루를 즐겁게 만들어줍니다 환한미소짓는하루되세요 은행,0
4,84766,2017-05,항상건강하시고사랑가득미소가득한행복한하루보내세요0 부산시청 올림,0


In [0]:
Counter(train_sample.smishing)

Counter({0: 74812, 1: 18703})

# Tokenizing

문장 분석을 위해 뜻을 가지고 있는 형태소들만 문장에서 추출한다.

문장을 형태소로 분해하기 위해 한국어 형태소 분석기인 mecab을 사용했다.

많은 한국어 형태소 분석기가 있지만 mecab이 성능과 속도 두 가지 측면에서 모두 좋아서 사용하게 되었다.(밑의 설치 코드는 코랩 환경(리눅스)에서 실행했다. 윈도우에서 실행하려면 다른 방법을 사용해야 한다.)

In [0]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 154kB/s 
     |████████████████████████████████| 2.4MB 37.2MB/s 
     |████████████████████████████████| 92kB 6.3MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [0]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Installing automake (A dependency for mecab-ko)
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [826 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [836 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [37.1 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/u

모든 형태소를 사용하지 않고 문장에서 의미가 있다고 판단한 품사만 사용했다.

In [0]:
from konlpy.tag import Mecab
mecab = Mecab()

In [0]:
text = '투자분'
mecab.morphs(text)

['투자', '분']

In [0]:
mecab.pos(text)

[('프랑스', 'NNP'),
 ('업체', 'NNG'),
 ('패럿', 'NNG'),
 ('이', 'JKS'),
 ('공개', 'NNG'),
 ('한', 'XSV+ETM'),
 ('드론', 'NNP')]

In [0]:
from konlpy.tag import Mecab
mecab = Mecab()

# NNG : 일반명사
# VA : 형용사
# VV : 동사
p = ['NNG', 'VA', 'VV']

# 입력값으로 받은 문장을 형태소로 분해한 뒤 리스트로 반환해주는 함수
def tokenizer(text, pos=p):
  return [word for word, tag in mecab.pos(text) if tag in pos]

train_sample['text'] = train_sample['text'].apply(tokenizer)
# test['text'] = test['text'].apply(tokenizer)

In [0]:
# 토크나이징 결과 확인
train_sample.head()

,id,year_month,text,smishing
0,191613,2017-10,"[계장, 주, 중반, 수요일, 햇살, 하루, 보내]",0
1,323818,2018-11,"[광고, 개인, 선택, 필수, 고객, 연말, 정산, 세액, 보너스, 연금, 수령, ...",0
2,224978,2018-02,"[고객, 활기, 넘치, 술년, 해, 건강, 보내, 고향, 정, 가족, 나누, 설명,...",0
3,73014,2017-04,"[웃, 얼굴, 하루, 즐겁, 만들, 미소, 짓, 하루, 은행]",0
4,84766,2017-05,"[건강, 사랑, 미소, 행복, 하루, 보내, 시청]",0


# 검증용 데이터 나누기

In [0]:
X = train_sample.text.values
y = train_sample.smishing.values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((74812,), (18703,), (74812,), (18703,))

# 다단어 표현 추출



'리브앱' 과 같이 사전에 등록되어 있지 않은 단어는 형태소 분석기가 인식하지 못하기 때문에 이러한 단어들을 한 단어로 묶기 위해 사용하였다.

In [0]:
%%time
bigrams = Phraser(Phrases(x_train))
trigrams = Phraser(Phrases(bigrams[x_train]))

CPU times: user 21.8 s, sys: 67.1 ms, total: 21.9 s
Wall time: 21.9 s


In [0]:
%%time
bi_train = bigrams[x_train]
bi_test = bigrams[x_test]

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 23.4 µs


In [0]:
%%time
tri_train = trigrams[bigrams[x_train]]
tri_test = trigrams[bigrams[x_test]]

CPU times: user 0 ns, sys: 277 µs, total: 277 µs
Wall time: 284 µs


In [0]:
x_train = np.concatenate((x_train, bi_train, tri_train))
y_train = np.concatenate((y_train, y_train, y_train))

# word2vec

모델의 학습 속도를 높이기 위하여 임베딩 층의 가중치로 word2vec 학습을 통해 얻은 단어의 분산표현을 사용했다.

In [0]:
%%time
embedding_vector_size = 100      # 임베딩 벡터의 차원

trigram_model = Word2Vec(
    sentences = x_train,
    size = embedding_vector_size,
    min_count = 5, window = 5, workers = 4
)

CPU times: user 1min 41s, sys: 496 ms, total: 1min 42s
Wall time: 55.2 s


In [0]:
print("vocabulrary size : ", len(trigram_model.wv.vocab))

vocabulrary size :  17718


In [0]:
trigram_model.wv.vocab

# 정수인코딩, 패딩

문장을 모델의 입력으로 넣으려면 벡터 형태로 바꿔야 한다.
훈련된 word2vec 모델의 사전을 이용하여 사전에 단어가 있는 경우 정수 인코딩을 수행하고 없는 경우 해당 단어를 삭제한다.
그리고 입력 데이터 차원은 모두 같아야하기 때문에 패딩을 이용하여 차원을 맞춰준다.

In [0]:
def vectorize_data(data, vocab: dict) -> list:
    keys = list(vocab.keys())
    filter_unknown = lambda word: vocab.get(word, None) is not None # 출력값은 논리값으로 나온다
    encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
    vectorized = list(map(encode, data))
    return vectorized
 
input_length = max([len(x) for x in x_train])

# train dataset
X_pad = pad_sequences(
    sequences=vectorize_data(x_train, vocab=trigram_model.wv.vocab),
    maxlen=input_length,
    padding='post')

# test dataset
test_pad = pad_sequences(
    sequences=vectorize_data(x_test, vocab=trigram_model.wv.vocab),
    maxlen=input_length,
    padding='post')

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

# 모델

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

In [0]:
def build_model(embedding_matrix: np.ndarray, input_length: int):
    model = Sequential()
    model.add(Embedding(
        input_dim = embedding_matrix.shape[0],
        output_dim = embedding_matrix.shape[1], 
        input_length = input_length,
        weights = [embedding_matrix],
        trainable=False, # 미리 word2vec로 학습된 단어벡터를 사용하기 때문에 훈련시키지 않는다.
        mask_zero=True)) # mask_zero를 사용하면 패딩한 부분은 패딩으로 인식하고 계산하지 않는다.
    model.add(LSTM(128, recurrent_dropout=0.1))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.summary()
    return model

model = build_model(
    embedding_matrix=trigram_model.wv.vectors,
    input_length=input_length)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 333, 100)          1768200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 130       
Total params: 1,893,834
Trainable params: 125,634
Non-trainable params: 1,768,200
______________________________________

In [0]:
adam = Adam(lr=0.0001)
auc = AUC()
model.compile(
    loss="categorical_crossentropy",
    optimizer=adam,
    metrics=['accuracy', auc])

cw = {0:0.3, 1:0.7}

history = model.fit(
    x=X_pad,
    y=y_train,
    validation_data=(test_pad, y_test),
    batch_size=256,
    epochs=1,
    class_weight = cw,
    shuffle=True
    )

Train on 224436 samples, validate on 18703 samples
224436/224436 [==============================] - 648s 3ms/sample - loss: 0.0493 - acc: 0.9551 - auc_4: 0.9922 - val_loss: 0.0109 - val_acc: 0.9970 - val_auc_4: 0.9997


eopch 1번으로도 좋은 결과가 나온다.